# Multi-Agent

This notebook demonstrates how you can use the Generative AI Toolkit to implement a multi-agent architecture.

In the example here, we'll build an agent whose job it is to give the user information on the city they want to travel to.

This agent, the supervisor, has two subordinate agents that it can hand tasks to:

- Weather agent: to get the weather forecast for the city
- Events agents: to get a list of upcoming concerts and other events in the city

We can implement the multi-agent architecture by simply registering subordinate agents as tools with a higher-level agent. To be able to use an agent as tool, the agent must have a `name` and a `description`. Providing an `input_schema` is optional, but can be used to signal to the supervisor agent that it should send certain fields to the subordinate agent.

In the example here we just have a 2-level hierarchy: one supervisor, and 2 subordinate agents. You could add more levels if you want; one of the subordinate agents could itself be a supervisor to its own set of subordinate agents.


In [ ]:
import textwrap

from generative_ai_toolkit.agent import BedrockConverseAgent
from generative_ai_toolkit.ui import traces_ui

## Weather agent


In [ ]:
def get_weather(city: str):
    """
    Gets the weather forecast for the provided city

    Parameters
    ---
    city : string
        The city to get the weather forecast for
    """
    return "Sunny"


weather_agent = BedrockConverseAgent(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    system_prompt="You provide the weather forecast for the specified city.",
    name="transfer_to_weather_agent",  # will be used as the name of the tool registration, in the supervisor agent
    description="Get the weather forecast for a city.",  # will be used as the description of the tool registration, in the supervisor agent
)
weather_agent.register_tool(get_weather)

## Events agent


In [ ]:
def get_events(city: str):
    """
    Gets upcoming events (concerts, festivals, etc) in the provided city

    Parameters
    ---
    city : string
        The city to get the upcoming events for
    """
    return [
        {"name": "Nirvana tribute band concert", "when": "Tomorrow"},
        {"name": "Food truck festival", "when": "Coming Saturday"},
        {
            "name": "Open Museum day -- all museums have free entry",
            "when": "Coming Sunday",
        },
    ]


events_agent = BedrockConverseAgent(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    system_prompt="You provide the upcoming events for the specified city.",
    name="transfer_to_events_agent",  # will be used as the name of the tool registration, in the supervisor agent
    description="Get the upcoming events for a city.",  # will be used as the description of the tool registration, in the supervisor agent
    input_schema={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the upcoming events for",
            }
        },
        "required": [
            "city"
        ],  # By providing this overriding input_schema, we instruct the supervisor agent to pass the `city` field explicitly
    },
)
events_agent.register_tool(get_events)

## Supervisor agent


In [ ]:
# You can likely improve your results, by explicitly telling the supervisor agent that it is part of a multi-agent architecture:
multi_agent_prompt_prefix = textwrap.dedent(
    """
    # Multi-Agent Supervisor Context

    You are the supervisor in a multi-agent system. Your role is to coordinate specialized agents to fulfill complex user requests. Each agent under your supervision is responsible for a distinct capability or domain.

    When appropriate, delegate sub-tasks to the relevant agents by providing them with focused instructions. Aggregate their responses and synthesize a final answer for the user.

    Do not reveal the existence of subordinate agents or that task delegation has occurred. Your responses should appear as a single, seamless reply from one intelligent assistant.
    """
).strip()

supervisor = BedrockConverseAgent(
    model_id="eu.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=textwrap.dedent(
        """
        {multi_agent_prompt_prefix}

        # Instruction

        You provide users current information on cities that they may want to visit.
        """
    )
    .format(multi_agent_prompt_prefix=multi_agent_prompt_prefix)
    .strip(),
)
supervisor.register_tool(weather_agent)
supervisor.register_tool(events_agent)

## Chat with the supervisor agent

The supervisor agent will use the subordinate agents to help the user:


In [ ]:
supervisor.reset()
for chunk in supervisor.converse_stream(
    "I want to visit Amsterdam. Tell me the weather and events please"
):
    print(chunk, end="")

## Tracing

### All traces from supervisor (includes subagent traces)

The supervisor's traces include all traces from subagent invocations:


In [ ]:
supervisor_ui = traces_ui(supervisor.traces)
supervisor_ui.launch()

### Accessing individual subagent traces

When agents are used as tools, each invocation gets its own **subcontext ID** to maintain separate conversational memory. This enables:

- **Parallel invocations**: The supervisor could call both agents simultaneously, each with separate memory
- **Sequential invocations without memory carryover**: Calling the weather agent twice won't leak context between invocations
- **Follow-up messages**: You can send additional messages to a specific subagent invocation by reusing its subcontext ID

To view traces for a specific subagent invocation, we need to:

1. Extract the subcontext ID from the supervisor's traces
2. Set the correct conversation ID and subcontext ID on the subagent
3. Then access the subagent's traces

First, let's add a helper function to extract subcontext IDs:


In [ ]:
from collections import defaultdict


def get_subagent_subcontext_ids(traces):
    """
    Extract subcontext IDs used for each subagent invocation from traces.

    Returns a dict mapping subagent name -> set of subcontext IDs used
    """
    result = defaultdict(set)

    for trace in traces:
        if (
            trace.attributes.get("ai.trace.type") == "tool-invocation"
            and "ai.tool.subagent.subcontext.id" in trace.attributes
            and "ai.tool.name" in trace.attributes
        ):
            subagent_name = trace.attributes["ai.tool.name"]
            subcontext_id = trace.attributes["ai.tool.subagent.subcontext.id"]
            result[subagent_name].add(subcontext_id)

    return result

Now let's access the weather agent's traces for this specific invocation:


In [ ]:
# Store the conversation ID
conversation_id = supervisor.conversation_id

# Extract subcontext IDs from the supervisor's traces
subcontext_ids = get_subagent_subcontext_ids(supervisor.traces)

# Get the specific subcontext_id used for weather_agent
weather_subcontext_id = subcontext_ids["transfer_to_weather_agent"].pop()

# Set the weather agent to the correct conversation and subcontext
weather_agent.set_conversation_id(conversation_id, subcontext_id=weather_subcontext_id)

# Now weather_agent.traces returns the correct traces for this invocation
print(f"Weather agent subcontext ID: {weather_subcontext_id}")
print(f"Number of traces: {len(weather_agent.traces)}")

weather_agent_ui = traces_ui(weather_agent.traces)
weather_agent_ui.launch()

And the events agent's traces:


In [ ]:
# Get the specific subcontext_id used for events_agent
events_subcontext_id = subcontext_ids["transfer_to_events_agent"].pop()

# Set the events agent to the correct conversation and subcontext
events_agent.set_conversation_id(conversation_id, subcontext_id=events_subcontext_id)

# Now events_agent.traces returns the correct traces for this invocation
print(f"Events agent subcontext ID: {events_subcontext_id}")
print(f"Number of traces: {len(events_agent.traces)}")

events_agent_ui = traces_ui(events_agent.traces)
events_agent_ui.launch()